In [2]:
# Extraction of the QOIs for the 3D modedel from the snapshpots
import vtk
import numpy as np
import math
from vtk.util import numpy_support as VN
from scipy.io import savemat
from IPython.display import clear_output

N = 70;
M = 7;

ODIR = r"C:\Users\utente\Riccardo\Documenti\Uni\2\Tesi\Simulazionefinale\snapshots_2/"

output_t = ["Ct_t", "SF", "Cv_t"]
par_t = ["Ct", "SF", "Cv"]

ind = 0 # 0: Ct_t; 1: SF; 2: Cv_t
Y = np.zeros([N*(M+1), 18])
niter=N*(M+1)

for i in range(1,niter + 1):
	a = 0

	print("Sample n."+str(i) + " " + par_t[ind])

	for timestep in range(1,172,10):

		i_file = ODIR + str(i) + '/' + output_t[ind] + str(timestep) + '.vtk'
		reader = vtk.vtkDataSetReader()
		reader.SetFileName(i_file)
		reader.Update()

		data = reader.GetOutput()
		point_data = data.GetPointData()
		x = point_data.GetArray(par_t[ind])
		values = VN.vtk_to_numpy(x)

		points = data.GetPoints()
		point_coords = np.array([points.GetPoint(k) for k in range(points.GetNumberOfPoints())])

		if(i==1):
			connectivity = []
			cells = data.GetCells()
			cells.InitTraversal()
			idList = vtk.vtkIdList()

			while cells.GetNextCell(idList):
				node_ids = [idList.GetId(j) for j in range(idList.GetNumberOfIds())]
				connectivity.append(node_ids)

		spatialaverage = 0.0

		for cell_indices in connectivity:
			if(i==1):
				cell_coords = point_coords[cell_indices]

				v1 = cell_coords[1] - cell_coords[0]
				v2 = cell_coords[2] - cell_coords[0]
				v3 = cell_coords[3] - cell_coords[0]
				volume = np.abs(np.dot(np.cross(v1, v2), v3)) / 6.0

			cell_value = np.mean(values[cell_indices])

			spatialaverage += cell_value * volume

		Y[i-1, a] = spatialaverage
		a = a + 1
		print(spatialaverage)
	clear_output(wait=False)
	




In [5]:
ct = Y
np.save("ct_volume_completo.npy", ct)

istante_fine_infusione = 61
istante_3h = 9

ct_media_temporale = np.mean(ct, axis = 1)
ct_fine_infusione = ct[:,6]
ct_finale =  ct[:, -1]
ct_media_3h = np.mean(ct[:, 0:9], axis = 1)

from scipy.io import savemat


savemat('ct_average_560_3D.mat', {'ct_average_560_3D': ct_media_temporale})
savemat('ct_infusion_560_3D.mat', {'ct_infusion_560_3D': ct_fine_infusione})
savemat('ct_3h_560_3D.mat', {'ct_3h_560_3D': ct_media_3h})
savemat('ct_final_560_3D.mat', {'ct_final_560_3D': ct_finale})